## Download the Data

In [1]:
import os
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import numpy as np

DOWNLOAD_URL = "https://github.com/fatimaezzahra-creator/Projet-ML/raw/refs/heads/main/datasets/adult.tgz"
DATASET_PATH = "datasets"

def fetch_data(data_url, data_path):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    tgz_path = os.path.join(data_path, "adult.tgz")
    urllib.request.urlretrieve(data_url, tgz_path)
    tgz_file = tarfile.open(tgz_path)
    tgz_file.extractall(path=data_path)
    tgz_file.close()    

fetch_data(DOWNLOAD_URL, DATASET_PATH)

ModuleNotFoundError: No module named 'pandas'

## Dataset Exploratory Analysis

### Analysis of Form

In [ ]:
#load the data
def load_data():
    csv_path = os.path.join(DATASET_PATH, "adult.data")
    return pd.read_csv(csv_path)

data = load_data()
data.info()

In [ ]:
import seaborn as sns
#Missing Data Visualization
sns.heatmap(data.isna(), cbar=False)

We can see that the graph is all dark, which means there is no missing values in the data .

### Analysis of Content

In [ ]:
#Target Distribution Analysis
data[" class"].value_counts(normalize=True)

In [ ]:
import seaborn as sns
# Numerical attribute
for col in data.select_dtypes("int64"):
    sns.displot(data[col])

In [ ]:
#Categorical attribute
import matplotlib.pyplot as plt
for col in data.select_dtypes("object"):
    sns.displot(data=data, x=col)
    plt.title(f" '{col}'", fontsize=16)
    plt.xlabel(col, fontsize=12)
    plt.ylabel("Nombre d'observations", fontsize=12)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
#copy of the data
df=data.copy()
# Supprimer les espaces des noms de colonnes et des valeurs
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Creating subsets based on the target variable
class_0=df[df["class"] == "<=50K"]
class_1=df[df["class"] ==">50K"]
combined_df = (pd.concat([class_0, class_1]))

In [ ]:
#relation target-age
import seaborn as sns
sns.histplot(
  data=combined_df ,
  x="age",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#relation target-education_num
sns.histplot(
  data=combined_df ,
  x="education-num",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#ralation target-education
sns.histplot(
  data=combined_df ,
  x="education",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
education_mapping = data.groupby(" education")[" education-num"].unique()
# Afficher le mapping pour vérifier la correspondance
for edu, edu_num in education_mapping.items():
    print(f"Education: {edu}, Education_Num: {edu_num}")

In [ ]:
#relation target-workclass
sns.histplot(
  data=combined_df ,
  x="workclass",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#relation target-sex
sns.histplot(
  data=combined_df ,
  x="sex",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#relation target-marital status
sns.histplot(
  data=combined_df ,
  x="marital-status",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#relation target-relationship
sns.histplot(
  data=combined_df ,
  x="relationship",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
education_mapping = data.groupby(" marital-status")[" relationship"].unique()

# Afficher le mapping pour vérifier la correspondance
for mrs, rshp in education_mapping.items():
    print(f"marital-status: {mrs}, relationship: {rshp}")

In [ ]:
#relation taget-fnlwgt
sns.histplot(
  data=combined_df ,
  x="fnlwgt",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
# relation target-race
sns.histplot(
  data=combined_df ,
  x="race",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");


In [ ]:
#relation target-native country
sns.histplot(
  data=combined_df ,
  x="native-country",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#relation target-occupation
sns.histplot(
  data=combined_df ,
  x="occupation",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#rlation target-capital gain
sns.histplot(
  data=combined_df ,
  x="capital-gain",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#ralation target-capitalloss
sns.histplot(
  data=combined_df ,
  x="capital-loss",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#Capital Features Combination
dff=df.copy()
combined_df["capital_features"] = dff["capital-gain"] - combined_df["capital-loss"]


In [ ]:
#relation target- hours per week
sns.histplot(
  data=combined_df ,
  x="hours-per-week",
  hue="class",
  stat="density",
  common_norm=False,
  palette="bright");

## Data Pre-Processing
### Train/Test Split

In [84]:
data[target_name].value_counts()

class
<=50K    24720
>50K      7841
Name: count, dtype: int64

We can see that the distribution of the target class is NOT balanced, so to create our train and test sets we can use a StratifiedShuffleSplit that will not only shuffle the instances but also preserve the proportions in the original dataset.

In [85]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_indexes, test_indexes in split.split(data, data[target_name]):
    train_set = data.iloc[train_indexes]
    test_set = data.iloc[test_indexes]

print("Proportions in the original dataset:", data[target_name].value_counts(normalize=True))
print("Proportions in the train set:", train_set[target_name].value_counts(normalize=True))
print("Proportions in the test set:", test_set[target_name].value_counts(normalize=True))

Proportions in the original dataset: class
<=50K    0.75919
>50K     0.24081
Name: proportion, dtype: float64
Proportions in the train set: class
<=50K    0.759175
>50K     0.240825
Name: proportion, dtype: float64
Proportions in the test set: class
<=50K    0.759251
>50K     0.240749
Name: proportion, dtype: float64


### Separate target from the features

In [86]:
data = train_set.drop(target_name, axis=1)
target = train_set[target_name].copy()

numerical_features = data.select_dtypes(include=np.number).columns.tolist()
categorical_features = data.select_dtypes(include=['object']).columns.tolist()

### Transformation of Text and Categorical Data
The feature `education` is the only one which implies some kind of order, so we can use an `OrdinalEncoder`.
The features `workclass`, `marital-status`, `relationship`, `race` and `sex` can all be handled by a `OneHotEncoder`.
The features `occupation` and `native-country` have very high cardinality. They will also be handled by a `OneHotEncoder` for now, but we will eventually find a better solution.

As for numerical features, we will only use a `StandardScaler`. 

In [87]:
# Transformation of Text and Categorical Data
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

ordinal_features = ["education"]
not_ordinal_features = categorical_features.copy()
not_ordinal_features.remove("education")

preprocessor = ColumnTransformer([
    ("categorical_ordinal", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), ordinal_features),
    ("categorical_not_ordinal", OneHotEncoder(handle_unknown="ignore"), not_ordinal_features),
    ("numerical", StandardScaler(), numerical_features)
])

data_prepared = preprocessor.fit_transform(data)

## Training Models

In [ ]:
from sklearn.linear_model import LogisticRegression